# iTunes API

In [1]:
import requests
import pandas as pd
from datetime import datetime

In [2]:
base_url = 'https://itunes.apple.com/search'
term = 'the beatles'
country = 'in'
url = f"{base_url}?term={term}&country={country}"

response = requests.get(url)
response.status_code

200

### creating the url manually with all the parameters is time consuming,so we can use the params argument

In [3]:
response = requests.get(base_url, params={'term': 'the beatles', 'country': 'in'})
response.status_code

200

### exploring the response

In [4]:
data = response.json()
data.keys()

dict_keys(['resultCount', 'results'])

In [5]:
print(json.dumps(data['results'][0], indent=4))

{
    "wrapperType": "track",
    "kind": "song",
    "artistId": 136975,
    "collectionId": 1440833098,
    "trackId": 1440834224,
    "artistName": "The Beatles",
    "collectionName": "1 (2015 Version)",
    "trackName": "Hey Jude",
    "collectionCensoredName": "1 (2015 Version)",
    "trackCensoredName": "Hey Jude (2015 Stereo Mix)",
    "artistViewUrl": "https://music.apple.com/in/artist/the-beatles/136975?uo=4",
    "collectionViewUrl": "https://music.apple.com/in/album/hey-jude-2015-stereo-mix/1440833098?i=1440834224&uo=4",
    "trackViewUrl": "https://music.apple.com/in/album/hey-jude-2015-stereo-mix/1440833098?i=1440834224&uo=4",
    "previewUrl": "https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview128/v4/73/ac/8c/73ac8cf8-56db-34c6-10e6-8ff93f81b3c6/mzaf_7985505075688381983.plus.aac.p.m4a",
    "artworkUrl30": "https://is4-ssl.mzstatic.com/image/thumb/Music118/v4/0e/9b/d1/0e9bd186-2bb4-af3f-f4d0-62feb7a556da/source/30x30bb.jpg",
    "artworkUrl60": "https://is4-ss

In [6]:
print(f"Number of results: {data['resultCount']}")

Number of results: 50


### defining limit parameter during the request 

In [7]:
response = requests.get(base_url, params={'term': 'the beatles', 'country': 'us', 'limit': 100})
print(response.status_code)
data = response.json()
print(f"Number of results: {data['resultCount']}")

200
Number of results: 100


### invalid requests

In [8]:
response = requests.get(base_url, params={'term': 'the beatles', 'country': 'in', 'media': 'hi'})
response.status_code

400

In [9]:
response.json()

{'errorMessage': 'Invalid value(s) for key(s): [mediaType]',
 'queryParameters': {'output': 'json',
  'callback': 'A javascript function to handle your search results',
  'country': 'ISO-2A country code',
  'limit': 'The number of search results to return',
  'term': 'A search string',
  'lang': 'ISO-2A language code'}}

### printing track names and release dates

In [10]:
response = requests.get(base_url, params={'term': 'ritviz', 'country': 'us', 'limit': 10})
print(response.status_code)
data = response.json()
print(f"Number of results: {data['resultCount']}")

200
Number of results: 10


In [11]:
df = pd.DataFrame(columns=['Track name', 'Release date'])

for result in data['results']:
    # get name and date from the json response
    name = result['trackName']
    
    # get only the date from this format '2017-11-27T12:00:00Z'
    date = result['releaseDate'].split('T')[0]
    
    # convert date to a datetime object
    date = datetime.strptime(date, '%Y-%m-%d')
    
    # add name and date as a row to the pandas dataframe
    index = len(df)
    df.loc[index] = [name, date]
    
# drop duplicates and sort based on release date
df = df.drop_duplicates().sort_values(by='Release date')

# format time in dd-mm-yyyy format for easy readability
df['Release date'] = df['Release date'].dt.strftime('%d-%m-%Y')

# hide index while displaying the dataframe
df.style.hide_index()

Track name,Release date
Udd Gaye (Bacardi House Party Sessions),27-11-2017
Jeet,29-10-2018
Barso,16-11-2018
Jeet 2.0,03-04-2019
Sage,30-06-2019
Liggi,23-12-2019
Chalo Chalein (feat. Seedhe Maut),23-12-2019
Thandi Hawa,10-04-2020
Roshni,19-06-2020
Raahi,31-07-2020


### exporting data 

In [12]:
response = requests.get(base_url, params={'term': 'Alan Walker', 'country': 'us'})
print(response.status_code)
data = response.json()
print(f"Number of results: {data['resultCount']}")

200
Number of results: 50


In [13]:
songs_df = pd.DataFrame(data['results'])
songs_df.head()

,wrapperType,kind,artistId,collectionId,trackId,artistName,collectionName,trackName,collectionCensoredName,trackCensoredName,...,discNumber,trackCount,trackNumber,trackTimeMillis,country,currency,primaryGenreName,isStreamable,collectionArtistId,collectionArtistName
0,track,song,1062085272,1196294554,1196294581,Alan Walker,Faded - EP,Faded,Faded - EP,Faded,...,1,4,1,212627,USA,USD,Dance,True,NaN,NaN
1,track,song,1062085272,944459315,944459697,Alan Walker,NCS: Uplifting,Fade,NCS: Uplifting,Fade,...,1,16,7,264014,USA,USD,Dance,True,4035426.0,Various Artists
2,track,song,1062085272,1300172044,1300172874,"Alan Walker, Noah Cyrus & Digital Farm Animals",All Falls Down - Single,All Falls Down,All Falls Down - Single,All Falls Down,...,1,1,1,199111,USA,USD,Dance,True,NaN,NaN
3,track,song,1062085272,966367678,966367681,Alan Walker,Spectre - Single,Spectre,Spectre - Single,Spectre,...,1,1,1,230635,USA,USD,Dance,True,NaN,NaN
4,track,song,1062085272,1202263583,1202263655,Alan Walker,Alone - Single,Alone,Alone - Single,Alone,...,1,3,1,161200,USA,USD,Dance,True,NaN,NaN


In [14]:
# export the dataframe to a csv file
songs_df.to_csv('alan_walker_songs.csv')